<a href="https://colab.research.google.com/github/crematorium-crm/test_memoire1/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os , glob
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.image import img_to_array, load_img
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data_dir='/content/drive/MyDrive/dataSet_ISIC'


# Chemins vers les dossiers d'entraînement et de test
train_dir = os.path.join(data_dir, 'Train')
test_dir = os.path.join(data_dir, 'Test')

In [3]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(180, 180, 3))

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary' )

Found 1768 images belonging to 2 classes.
Found 195 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [6]:
#from PIL import Image

#def load_data(data_dir):
 #   images = []
  #  labels = []
  #  for folder in os.listdir(data_dir):
   #     label = 1 if folder == 'malin' else 0
    #    for image_file in os.listdir(os.path.join(data_dir, folder)):
      #      image = Image.open(os.path.join(data_dir, folder, image_file))
       #     image = image.resize((180, 180))
        #    image = np.array(image)
         #   images.append(image)
          #  labels.append(label)
    #return np.array(images), np.array(labels)

#train_images, train_labels = load_data(train_dir)
#test_images, test_labels = load_data(test_dir)

# Séparation des données d'entraînement en ensembles d'entraînement et de validation
#train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)

In [7]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

x = base_model.output
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [8]:
for layer in base_model.layers:
    layer.trainable = False
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

# Chargement des données d'entraînement et de validation
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(180, 180),
    batch_size=32,
    class_mode='binary' )

# Extraction des caractéristiques pour les ensembles d'entraînement et de validation
train_features = []
val_features = []

#for images, labels in train_generator:
#  features = model.predict(images)
 # train_features.append(features)

#for images, labels in val_generator:
 # features = model.predict(images)
  #val_features.append(features)

# Concaténation des features
#train_features = np.concatenate(train_features)
#val_features = np.concatenate(val_features)


Found 1768 images belonging to 2 classes.
Found 195 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [11]:

train_features = base_model.predict(train_generator, verbose=1)
val_features = base_model.predict(val_generator, verbose=1)

# Extraction des caractéristiques pour l'ensemble de test
test_features = base_model.predict(test_generator, verbose=1)

4/4 [==============================] - 22s 5s/step


In [12]:
print(np.unique(train_labels, return_counts=True))


NameError: name 'train_labels' is not defined

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# Aplatir les caractéristiques d'entraînement et de validation
train_features_flat = train_features.reshape(train_features.shape[0], -1)
val_features_flat = val_features.reshape(val_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Récupération des labels d'entraînement, de validation et de test
train_labels = np.array(train_generator.classes)
val_labels = np.array(val_generator.classes)
test_labels = np.array(test_generator.classes)
kernerl = 'rbf'


clf = SVC(kernel=kernerl, probability=True, verbose=0)  # Set verbose to 0 to avoid default output

# Nombre d'époques
epochs = 10

# Nombre d'échantillons à utiliser pour l'entraînement à chaque itération
batch_size = 64


for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Mélanger les données d'entraînement
    indices = np.random.permutation(len(train_features_flat))
    train_features_shuffled = train_features_flat[indices]
    train_labels_shuffled = train_labels[indices]

    # Entraîner le classifieur SVM par lots avec affichage de la progression
    for i in tqdm(range(0, len(train_features_flat), batch_size), desc=f"Epoch {epoch + 1} - Training"):
        end_index = min(i + batch_size, len(train_features_flat))
        _ = clf.fit(train_features_shuffled[i:end_index], train_labels_shuffled[i:end_index])

    # Évaluation sur les données de validation
    val_preds = clf.predict(val_features_flat)
    val_acc = accuracy_score(val_labels, val_preds)
    print(f"Validation accuracy: {val_acc:.4f}")

# Évaluation sur les données de test après toutes les époques
test_preds = clf.predict(test_features_flat)
test_acc = accuracy_score(test_labels, test_preds)
print(f"Test accuracy: {test_acc:.4f}")


Epoch 1/10


Epoch 1 - Training: 100%|██████████| 28/28 [00:48<00:00,  1.72s/it]


Validation accuracy: 0.5077
Epoch 2/10


Epoch 2 - Training: 100%|██████████| 28/28 [00:49<00:00,  1.76s/it]


Validation accuracy: 0.5231
Epoch 3/10


...

Validation accuracy: 0.4923
Epoch 9/10


Epoch 9 - Training: 100%|██████████| 28/28 [00:47<00:00,  1.70s/it]


Validation accuracy: 0.5077
Epoch 10/10


Epoch 10 - Training: 100%|██████████| 28/28 [00:50<00:00,  1.79s/it]


Validation accuracy: 0.5077
Test accuracy: 0.4435
